In [ ]:
!pip install transformers seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=b0d3f0e4eb2a0cc3887e24920de505eb4b86d9103af787032fab35b30b7d1ed8
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/AmazonML/student_resource/src/sanity.py /content
!cp /content/drive/MyDrive/AmazonML/student_resource/src/utils.py /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/sample_test.csv /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/sample_test_out.csv /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/sample_test_out_fail.csv /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/test.csv /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/train.csv /content
!cp /content/drive/MyDrive/AmazonML/student_resource/src/constants.py /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/final_train.csv /content
!cp /content/drive/MyDrive/AmazonML/student_resource/dataset/ocr_data.csv /content

In [ ]:
train_csv_path = '/content/final_train.csv'
test_csv_path = '/content/test.csv'
sample_test_csv_path = '/content/sample_test.csv'
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)
sample_test = pd.read_csv(sample_test_csv_path)

In [ ]:
train_df.head()

,image_link,group_id,entity_name,entity_value,ocr_data
0,https://m.media-amazon.com/images/I/715KM2mExJ...,397856,item_weight,1.0 ounce,"['Sturdy Eneugh to hold a 500g Welcht', 'By on..."
1,https://m.media-amazon.com/images/I/61vZIL-iEA...,120219,maximum_weight_recommendation,0.4 kilogram,"['BPA FREE', ' CERAMIC TUMBLER ', ' GOLD STIRR..."
2,https://m.media-amazon.com/images/I/81YTF2ehpn...,120219,maximum_weight_recommendation,4.0 ton,"['NEW!', 'SUPERBLENDS', 'Nightly', 'Calm®', 'S..."
3,https://m.media-amazon.com/images/I/81vbCQsHeC...,192132,item_weight,1.0 gram,"['SUPERBLENDS', 'NEW!', 'ATTATACE.', 'Nightly ..."
4,https://m.media-amazon.com/images/I/71qGT9euDC...,523149,item_weight,2.0 gram,"['Drink in Life', 'TWININGS', 'SUPERBLENDS', '..."


In [ ]:
print("Number of tags: {}".format(len(train_df.entity_name.unique())))
frequencies = train_df.entity_name.value_counts()
frequencies

Number of tags: 8


,count
entity_name,
item_weight,41261
voltage,3540
wattage,3015
item_volume,2754
maximum_weight_recommendation,1422
depth,63
height,56
width,51


In [ ]:
label2id = {k: v for v, k in enumerate(train_df.entity_name.unique())}
id2label = {v: k for v, k in enumerate(train_df.entity_name.unique())}
label2id

{'item_weight': 0,
 'maximum_weight_recommendation': 1,
 'voltage': 2,
 'item_volume': 3,
 'wattage': 4,
 'depth': 5,
 'height': 6,
 'width': 7}

In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
from transformers import BertTokenizer

# Example input sentences
data = [
    "This box weighs 500 grams and is 20 centimeters tall",
    "The product uses 1500 watts and operates at 220 volts"
]

# Entity unit map (from your input)
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','cm','ft','in','m','mm','yd'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','cm','ft','in','m','mm','yd'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard','cm','ft','in','m','mm','yd'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton','g','kg','μg','mg','oz','lb','tn'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton','g','kg','μg','mg','oz','lb','tn'},
    'voltage': {'kilovolt', 'millivolt', 'volt','kV','mV','V'},
    'wattage': {'kilowatt', 'watt','kW','W'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart',
                    'cl', 'cu ft', 'ft³', 'cu in', 'in³', 'cup', 'dl', 'fl oz', 'gal', 'imp gal', 'L', 'µL', 'mL', 'pt', 'qt'}
}

# Allowed units
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}


In [ ]:
import pandas as pd
from transformers import BertTokenizer
import re

# Helper function to extract value and unit from entity_value
def extract_value_and_unit(entity_value):
    # Split into number and unit using regular expressions
    match = re.match(r"(\d+)\s*(.*)", entity_value)
    if match:
      value = match.group(1)  # The numeric part
      unit = match.group(2).strip().lower()  # The unit part
      return value, unit
    return None, None

# Function to tokenize and label ocr_data based on entity_value
def tokenize_and_label(data_list, entity_value, entity_name):
    sentence = ""
    for data in data_list:
        if data != "[" and data != "]" and data!="'" and data!=",":
          sentence += data

    tokens = tokenizer.tokenize(sentence)
    labels = ['O' for _ in tokens ]
    # Extract the numeric value and unit from entity_value
    expected_value, expected_unit = extract_value_and_unit(entity_value)

    i = 0
    while i < len(tokens):
        token = tokens[i]

        # Check if the token is a number and matches the expected value
        if token == expected_value:
            labels.append(f"B-{entity_name}")  # Label the value as B-{entity}
            i += 1  # Move to the next token
            found = False
            j=0
            while j < len(tokens):
                token = tokens[j]
                if token in allowed_units:
                    found = True
                    j+=1
                    break
                else:
                  found=False
                  j+=1
            if found==True:
                labels.append(f"I-{entity_name}")  # Label the unit as I-{entity}
            else:
                i += 1  # Move to the next token

        else:
            i += 1  # Move to the next token

    return labels

# Function to process the ocr_data column and return labels
def process_column(df):
    label_list = []

    for idx, row in df.iterrows():
        ocr_data = row['ocr_data']
        entity_value = row['entity_value']
        entity_name = row['entity_name']

        # Get the tokenized and labeled result
        labels = tokenize_and_label(ocr_data, entity_value, entity_name)
        label_list.append(",".join(labels))  # Join labels with a comma
        print("Row", idx )
        print("Completed")

    return label_list


In [ ]:
train_df_ex=train_df.head(20)
# Apply the tokenization and labeling to the 'ocr_data' column
train_df_ex['ocr_data_labels'] = process_column(train_df_ex)

# Display the resulting DataFrame with the formatted 'ocr_data_labels'
print(train_df_ex[['ocr_data', 'ocr_data_labels']])

Row 0
Completed
Row 1
Completed
Row 2
Completed
Row 3
Completed
Row 4
Completed
Row 5
Completed
Row 6
Completed
Row 7
Completed
Row 8
Completed
IF 2 is EXECUTED
IF 2 is EXECUTED
IF 2 is EXECUTED
Row 9
Completed
Row 10
Completed
Row 11
Completed
Row 12
Completed
Row 13
Completed
Row 14
Completed
Row 15
Completed
Row 16
Completed
Row 17
Completed
Row 18
Completed
Row 19
Completed
                                             ocr_data  \
0   ['Sturdy Eneugh to hold a 500g Welcht', 'By on...   
1   ['BPA FREE', ' CERAMIC TUMBLER ', ' GOLD STIRR...   
2   ['NEW!', 'SUPERBLENDS', 'Nightly', 'Calm®', 'S...   
3   ['SUPERBLENDS', 'NEW!', 'ATTATACE.', 'Nightly ...   
4   ['Drink in Life', 'TWININGS', 'SUPERBLENDS', '...   
5   ['NEW!', 'SUPERBLENDS', 'LONDON', 'Nightly\nCa...   
6   ['NEW! ', 'SUPERBLENDS', 'LONDON .', 'Soothing...   
7   ['Product Parameters', 'Size: 60x37x8.5mm', 'W...   
8   ['Puppy', 'Aircraft', 'Turtle', '2D Model To 3...   
9   ['BEST HAD AS BLACK', 'res', '100% pure Arabi

<ipython-input-101-fa26118670db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_ex['ocr_data_labels'] = process_column(train_df_ex)


In [ ]:
print(train_df[['ocr_data', 'ocr_data_labels']].iloc[504])

ocr_data           ['Waterproof', '=E26', '', '100W', '120V', 'AZ...
ocr_data_labels    O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,...
Name: 504, dtype: object
